In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d
from tslearn.preprocessing import TimeSeriesResampler
from Process_Functions import Bessel_Filter, Down_sampling, normalized_translocation, interplotation, Draw_Plot, normalized_time_axis, normalized_delta
from scipy.ndimage import gaussian_filter1d
import os

In [7]:
def Data_Processing(file_name):
    if file_name[-3:]=='lsx':
        original_data=pd.read_excel(file_name)
        
    elif file_name[-3:]=='csv':
        original_data=pd.read_csv(file_name)
    original_data = original_data.filter(regex='^(?!Unnamed)')
    signal_index = [int(col) for n, col in enumerate(original_data.columns) if (n + 1) % 3 == 1]  
    data_cleared=original_data.dropna(axis=1,how='all')
    #slice different serieses
    data_sliced = []
    column_num = len(data_cleared.columns)

    for i in range(0, column_num, 2):
        df_sliced = data_cleared.iloc[:, i:i+2].copy()  
        df_sliced.columns = ['t', 'i' ]
        # 删除所有值为空的行
        new_df_sliced = df_sliced.dropna(axis=0, how="all").copy()
        # 确保 new_df_sliced 不是空的再进行操作
        if not new_df_sliced.empty:
            new_df_sliced.loc[:, new_df_sliced.columns[0]] -= new_df_sliced.iloc[0, 0]
        data_sliced.append(new_df_sliced)
    return data_sliced

In [8]:
'''
file_name:导入文件的名称 默认为多列时间-电流数据（参考1.xlsx中的格式）
start end 指定导入数据的范围 默认全部
upper lim lower lim 决定数据长度范围

sampling frequency:导入数据的采样频率 默认100(KHz)
cut_off_frequency:截止频率，决定滤波的程度默认2(KHz)
drop head drop_tail 去掉处理后数据的前后几个数据点（由于采用的分离过孔信号方法，前后会产生很高的数据点）默认去掉5个数据点
smooth:是否使用高斯滤波器平滑数据，默认否
normalization_method 决定是-均值/标准差‘standard’还是‘delta’（deltaI/I_0,默认为standard）
'''
def Signal_Processing(file_name,start=None, end=None, upper_lim=None,lower_lim=1,sampling_frequency=100,cut_off_frequency=2,
                      drop_head=5,drop_tail=-5,smooth=False, normalization_method='standard',threshold=60):
    if file_name[-3:]=='lsx':
        original_data=pd.read_excel(file_name)
        
    elif file_name[-3:]=='csv':
        original_data=pd.read_csv(file_name)
    original_data = original_data.filter(regex='^(?!Unnamed)')
    signal_index = [int(col) for n, col in enumerate(original_data.columns) if (n + 1) % 3 == 1]  
    data_cleared=original_data.dropna(axis=1,how='all')
    #slice different serieses
    data_sliced = []
    column_num = len(data_cleared.columns)

    for i in range(0, column_num, 2):
        df_sliced = data_cleared.iloc[:, i:i+2].copy()  
        df_sliced.columns = ['t', 'i' ]
        # 删除所有值为空的行
        new_df_sliced = df_sliced.dropna(axis=0, how="all").copy()
        # 确保 new_df_sliced 不是空的再进行操作
        if not new_df_sliced.empty:
            new_df_sliced.loc[:, new_df_sliced.columns[0]] -= new_df_sliced.iloc[0, 0]
        data_sliced.append(new_df_sliced)

        
    #process the data
    #set parameters
    processed_data=[]
    for series in data_sliced:
        filtered_current=Bessel_Filter(series['i'],sampling_frequency=sampling_frequency,cut_off_frequency=cut_off_frequency)
        filtered_curve=np.array([series['t'],filtered_current])
        down_sampled_curve=filtered_curve
        filtered_curve_dataframe=pd.DataFrame(down_sampled_curve.T,columns=['t','i'])
        if normalization_method=='standard':
            normalized_data=normalized_translocation(filtered_curve_dataframe,threshold=min(filtered_curve_dataframe.iloc[:,1])+threshold,drop_head=drop_head, drop_tail=drop_tail)
            processed_data.append(normalized_data)
        elif normalization_method=='delta':
            normalized_data=normalized_delta(filtered_curve_dataframe,threshold=min(filtered_curve_dataframe.iloc[:,1])+threshold,drop_head=drop_head, drop_tail=drop_tail)
            processed_data.append(normalized_data)

    #select data for further alignment
    if upper_lim is None:
        target_data=processed_data
   
    else:
        target_data=[series for series in processed_data if len(series)<upper_lim and len(series)>lower_lim]
        selected_index=[]
        for index, series in enumerate(processed_data):
            if len(series) < upper_lim and len(series) > lower_lim:
                selected_index.append(index)
        selected_signal_index=[signal_index[i] for i in selected_index]
        
                
    
    #get the length of each selected signal and 
    len_target_data=[len(series) for series in target_data]
    #max_len=max(len_target_data)
    #max_index=[len(series) for series in target_data].index(max_len) 

    #using time_axis normalization method to normallize the time 
    if smooth==True:
        for series in target_data:
            series['i']=gaussian_filter1d(series['i'].values,sigma=3)

    
    time_normalized_data=[normalized_time_axis(series) for series in target_data][start:end]

    
    return time_normalized_data,selected_signal_index

In [9]:
def export_data(data,save_path='',file_name='1',signal_index=None):
    data_processed=[]
    if signal_index==None:
        for i in range(len(data)):
            series_index=pd.DataFrame([], columns=[str(i+1)])
            data_processed.append(series_index)
            if isinstance(data[0],np.ndarray):
                print(data[0].ndim==1)
                if data[0].ndim==1:
                    data_processed.append(pd.DataFrame({'t':np.linspace(0,1,len(data[i])),'i':data[i]}))
                elif data[0].ndim==2:
                    data_processed.append(pd.DataFrame(data[i].T,columns=['t','i']))
                else:
                    print('TypeError, Please check data type')
            elif isinstance(data[0],pd.DataFrame):
                data_processed.append(data[i])
            else:
                print('TypeError, Please check data type')
            
    else:
        for i,ind in enumerate(signal_index):
            series_index=pd.DataFrame([], columns=[str(ind)])
            data_processed.append(series_index)
            
            if isinstance(data[0],np.ndarray):
                if data[0].ndim==1:
                    data_processed.append(pd.DataFrame({'t':np.linspace(0,1,len(data[i])),'i':data[i]}))
                elif data[0].ndim==2:
                    data_processed.append(pd.DataFrame(data[i].T,columns=['t','i']))
                else:
                    print('TypeError, Please check data type')
            elif isinstance(data[0],pd.DataFrame):
                data_processed.append(data[i])
            else:
                    print('TypeError, Please check data type')

            
            
    
#merge all the data 
    max_length = max([df.shape[0] for df in data_processed])

    # 填充每个 DataFrame 至最大行数
    for i, df in enumerate(data_processed):
        if df.shape[0] < max_length:
            num_missing = max_length - df.shape[0]
            missing_df = pd.DataFrame(index=range(num_missing), columns=df.columns)
            data_processed[i] = pd.concat([df, missing_df], ignore_index=True)

    # 水平拼接所有 DataFrame
    combined_df = pd.concat(data_processed, axis=1)

    # 导出为 Excel 文件
    cur_dir=os.getcwd()
    abs_save_path=os.path.join(cur_dir,save_path)
    if not os.path.exists(abs_save_path):
        os.makedirs(abs_save_path)
    combined_df.to_excel(os.path.join(abs_save_path,file_name+'.xlsx'), index=False)

In [10]:
current_dir=os.getcwd()
dir_120='data_120'
dir_final='data_final'
dir_new='data_new'
direction_1='direction_1.xlsx'
direction_2='direction_2.xlsx'


data_sliced_1=Data_Processing(os.path.join(current_dir,dir_new,direction_1))
data_sliced_2=Data_Processing(os.path.join(current_dir,dir_new,direction_2))



In [11]:
current_dir=os.getcwd()
dir_120='data_120'
dir_final='data_final'
dir_new='data_new'
direction_1='direction_1.xlsx'
direction_2='direction_2.xlsx'

In [16]:
time_normalized_data_1,selected_signal_index_1=Signal_Processing(os.path.join(current_dir,dir_new,direction_1),upper_lim=200000,lower_lim=1,
                                        cut_off_frequency=5,normalization_method='standard',smooth=False,threshold=60)
time_normalized_data_2,selected_signal_index_2=Signal_Processing(os.path.join(current_dir,dir_new,direction_2),upper_lim=200000,lower_lim=1,
                                        cut_off_frequency=5,normalization_method='standard',smooth=False)

In [13]:
processed_data_1=[]
for series in data_sliced_1:
    filtered_current=Bessel_Filter(series['i'],sampling_frequency=100,cut_off_frequency=2)
    filtered_curve=np.array([series['t'],filtered_current])
    down_sampled_curve=filtered_curve
    filtered_curve_dataframe=pd.DataFrame(down_sampled_curve.T,columns=['t','i'])
    processed_data_1.append(filtered_curve_dataframe)

In [14]:
processed_data_2=[]
for series in data_sliced_2:
    filtered_current=Bessel_Filter(series['i'],sampling_frequency=100,cut_off_frequency=5)
    filtered_curve=np.array([series['t'],filtered_current])
    down_sampled_curve=filtered_curve
    filtered_curve_dataframe=pd.DataFrame(down_sampled_curve.T,columns=['t','i'])
    processed_data_2.append(filtered_curve_dataframe)

In [19]:
export_data(processed_data_1,os.path.join(current_dir,'filtered_data'),'direction_1',signal_index=selected_signal_index_1)
export_data(processed_data_2,os.path.join(current_dir,'filtered_data'),'direction_2',signal_index=selected_signal_index_2)


C:\Users\yjsun\AppData\Local\Temp\ipykernel_7908\475777862.py:48: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  data_processed[i] = pd.concat([df, missing_df], ignore_index=True)
C:\Users\yjsun\AppData\Local\Temp\ipykernel_7908\475777862.py:48: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  data_processed[i] = pd.concat([df, missing_df], ignore_index=True)
C:\Users\yjsun\AppData\Local\Temp\ipykernel_7908\475777862.py:48: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA 